In [2]:
%load_ext autoreload
%autoreload 2


In [3]:

from proj.cxs_are_revealed.paper.data_config import Exp3Magpie
from paper.exp3_magpie.corpus_magpie import get_all_magpie_json, get_magpie_pretty, set_allow_deprecated

set_allow_deprecated()
all_magpie_json = get_all_magpie_json()
all_magpie_entries, all_magpie_wrappers = get_magpie_pretty(all_magpie_json)


/opt/homebrew/Caskroom/miniconda/base/envs/2024_coelm/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1538 > 512). Running this sequence through the model will result in indexing errors


In [9]:
from paper.exp3_magpie.corpus_magpie import MAGPIE_entry, MagpieEntryForProcessing
##############
#### produce magpie for use on cluster
##############

from typing import List

def make_magpie_json_for_cluster(magpie_entries: List[MAGPIE_entry]):
    all_json: List[MagpieEntryForProcessing] = []
    for idx, me in enumerate(magpie_entries):
        assert idx == me.id
        all_json.append(
            MagpieEntryForProcessing(
                me.id,
                me.sent
            )
        )
    return all_json

magpie_json_for_cluster = make_magpie_json_for_cluster(all_magpie_entries)


In [11]:
from rozlib.libs.common.data.utils_jsonl import dump_to_jsonl

# write into files
# magpie-unclean - will have extra spaces (i.e. no tokenization clean up)
# magpie_dir = Path("../../../../../data/corpus_parsed/magpie_unclean")
magpie_dir = Exp3Magpie.magpie_text_file_dir
chunksize = 1000

def write_to_chunked_files(all_inputs: List[MagpieEntryForProcessing]):
    line_ct = 0
    file_ct = 0
    # f = open(magpie_dir / f"{file_ct}.txt", "w")
    f = magpie_dir / f"{file_ct}.txt"

    for idx, i in enumerate(all_inputs):
        if line_ct == chunksize:
            # f.close()
            line_ct = 0
            file_ct += 1
            # f = open(magpie_dir / f"{file_ct}.txt", "w")
            f = magpie_dir / f"{file_ct}.txt"
        dump_to_jsonl(i, f)
        # f.write(i + "\n")
        line_ct += 1
    # f.close()

# uncomment to rewrite output
# write_to_chunked_files(magpie_json_for_cluster)
